# Capstone Project - The Battle of the Neighborhoods (Week 1)

## Introduction: Business Problem

Cafe ABC has been operating in Rosemead, CA since 2018. The cafe serves iconic Taiwanese dishes such as popcorn chicken, beef noodle soup, stinky tofu, minced meat over rice, and various flavors of boba drinks. The owners are considering opening a second location in Southern California due to the popularity of the first branch.

Some factors to consider in choosing a second location are: locations that don't have a high concentration of restaurants to avoid heightened competition; demographics of the neighborhood such that introducing a Taiwanese eatery won't be too unfamiliar.

We will leverage data science to identify a few cities that are suitable as a second location. This report will be useful for stakeholders interested in opening a Taiwanese restaurant or cafe in the Southern California region.

## Data

We will define our area of interest to be cities in Los Angeles county in Southern California, which is the county that the original location belongs to. Based on the definition of our problem, factors that will influence the decision will include:

* How similar the demographics is compared to Rosemead, CA, representing how likely an Asian eatery will be successful cultural-wise
* Number of existing restaurants in the same city

The following data sources will be used to extract and generate the required information:

* Geospatial data for cities in the LA county will be sourced from Los Angeles GeoHub, an open-source data hub for location-based data
* Demographics data will be will be obtained by webscrapping sites such as Wikipedia, which is ultimately sourced from United States Census Bureau
* Number of restaurants and their types and locations will be obtained using Foursquare API

### Geospatial Data

First we install the necessary packages.

In [1]:
!pip install bs4
from bs4 import BeautifulSoup # to help with webscrapping

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Imports and installations done.')

Imports and installations done.


Since our goal is to identify cities suitable for a second location for the cafe, we need to obtain a list of cities in the Los Angeles county, along with their geospatial data such as longitudes and latitudes. Los Angeles GeoHub is an online portal with public access to the city's location-based data. One of the datasets include a list of city halls in LA county. We will extract the city name, zip code, longitude, and latitude information from this dataset.

In [2]:
!wget -q -O 'geo_data.json' https://opendata.arcgis.com/datasets/db2c52f3ddc945cb988c393deac1d487_67.geojson
print('Data downloaded!')

Data downloaded!


In [3]:
with open('geo_data.json') as json_data:
    losangeles_data = json.load(json_data)

We see that within this dataset, the relevant data (city, zip code, latitude, longitude) are all in the features key, so we define a new variable for this data.

In [4]:
geospatial_data = losangeles_data['features']
geospatial_data[0]

{'type': 'Feature',
 'properties': {'OBJECTID': 2576,
  'source': 'City of Manhattan Beach',
  'ext_id': '',
  'cat1': 'Government',
  'cat2': 'City Halls',
  'cat3': None,
  'org_name': 'City of Manhattan Beach',
  'Name': 'City Of Manhattan Beach',
  'addrln1': '1400 Highland Ave',
  'addrln2': None,
  'city': 'Manhattan Beach',
  'state': 'CA',
  'hours': None,
  'phones': 'FAX (310) 802-5001,  Service/Intake and Administration (310) 802-5000, City Clerk Service/Intake (310) 802-5056, Permits Service/Intake (310) 802-5536, City Attorney Service/Intake (310) 802-5061, Business Licenses Service/Intake (310) 802-5558, Permits S',
  'url': 'http://www.citymb.info',
  'info1': None,
  'info2': None,
  'post_id': 2815,
  'description': '',
  'zip': '90266',
  'link': 'http://egis3.lacounty.gov/lms/?p=2815',
  'use_type': 'publish',
  'latitude': 33.88728147,
  'longitude': -118.41060698,
  'date_updated': '2011-02-09T11:08:51Z',
  'email': None,
  'dis_status': None,
  'POINT_X': 6437047.

We now want to transform the json data into a *pandas* dataframe.

In [5]:
# define dataframe columns
column_names = ['City','ZipCode','Latitude','Longitude']

# instantiate the dataframe
cities = pd.DataFrame(columns=column_names)

In [6]:
# loop through the data and fill the dataframe one row at a time
for data in geospatial_data:
    city = data['properties']['city'] 
    zipcode = data['properties']['zip']
    latitude = data['properties']['latitude']
    longitude = data['properties']['longitude']
    
    cities = cities.append({'City': city,
                            'ZipCode': zipcode,
                            'Latitude': latitude,
                            'Longitude': longitude}, ignore_index=True)

In [7]:
# delete the row with index 4 since it has no value
cities = cities.drop([4],axis=0)

In [8]:
cities.head()

,City,ZipCode,Latitude,Longitude
0,Manhattan Beach,90266,33.887281,-118.410607
1,Cerritos,90703,33.867227,-118.063873
2,Claremont,91711,34.095726,-117.716532
3,Burbank,91502,34.181820,-118.307890
5,Agoura Hills,91301,34.144303,-118.777612


We now have a dataframe with city names, zip code, longitudes and latitudes.

In [9]:
cities.shape

(91, 4)

### Demographics Data

We will obtain demographics data by webscrapping a Wikipedia page.

In [10]:
#The below url contains html tables with data on cities in Los Angeles county
url = "https://en.wikipedia.org/wiki/Demographics_of_Los_Angeles_County"
data  = requests.get(url).text
soup = BeautifulSoup(data,"html5lib")

In [11]:
#find all html tables in the web page
tables = soup.find_all('table')
len(tables)

4

We create a dataframe and loop through the data to fill in each row.

In [12]:
demographics_data = pd.DataFrame(columns=["City","Total Population","White","African American","Native American","Asian","Pacific Islander","Other","Two or More","Hispanic"])

for row in tables[3].tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        city = col[0].text.replace('\n','')
        population = col[1].text.replace(',','')
        white = col[2].text.replace(',','')
        african_american = col[3].text.replace(',','')
        native_american = col[4].text.replace(',','')
        asian = col[5].text.replace(',','')
        pacific_islander = col[6].text.replace(',','')
        other = col[7].text.replace(',','')
        two_or_more = col[8].text.replace(',','')
        hispanic = col[9].text.replace(',','').replace('\n','')
        demographics_data = demographics_data.append({"City": city, "Total Population": population, "White": white,"African American": african_american,
                                                      "Native American": native_american,"Asian": asian,"Pacific Islander": pacific_islander,"Other": other,
                                                      "Two or More": two_or_more,"Hispanic":hispanic},ignore_index=True)

In [13]:
demographics_data[0:10]

,City,Total Population,White,African American,Native American,Asian,Pacific Islander,Other,Two or More,Hispanic
0,The County,TotalPopulation,White,AfricanAmerican,NativeAmerican,Asian,PacificIslander,otherraces,two ormore races,Hispanicor Latino(of any race)
1,Los Angeles County,9818605,4936599,856874,72828,1346865,26094,2140632,438713,4687889
2,,100%,50.3%,8.7%,0.7%,13.7%,0.3%,21.8%,4.5%,47.7%
3,Incorporatedcity,TotalPopulation,White,AfricanAmerican,NativeAmerican,Asian,PacificIslander,otherraces,two ormore races,Hispanicor Latino(of any race)
4,Agoura Hills,20330,17147,267,51,1521,24,590,730,1936
5,Alhambra,83089,23521,1281,538,43957,81,10805,2906,28582
6,Arcadia,56364,18191,681,186,33353,16,2352,1585,6799
7,Artesia,16522,6446,589,94,6131,40,2630,592,5910
8,Avalon,3728,2313,20,22,49,13,1137,174,2079
9,Azusa,46361,26715,1499,562,4054,87,11270,2174,31328


We see that the data for each city begins with row with index 4, and there are a few rows with sub-headers that we should drop.

In [14]:
demographics_data = demographics_data.drop([0,1,2,3,92,146,147])
demographics_data[0:5]

,City,Total Population,White,African American,Native American,Asian,Pacific Islander,Other,Two or More,Hispanic
4,Agoura Hills,20330,17147,267,51,1521,24,590,730,1936
5,Alhambra,83089,23521,1281,538,43957,81,10805,2906,28582
6,Arcadia,56364,18191,681,186,33353,16,2352,1585,6799
7,Artesia,16522,6446,589,94,6131,40,2630,592,5910
8,Avalon,3728,2313,20,22,49,13,1137,174,2079


In order to compare among the cities, it's better to convert the absolute number of each demographic to a ratio. First we need to convert every column except for the City column to integers. 

In [15]:
demographics_data["Total Population"] = pd.to_numeric(demographics_data["Total Population"])
demographics_data["White"] = pd.to_numeric(demographics_data["White"])
demographics_data["African American"] = pd.to_numeric(demographics_data["African American"])
demographics_data["Native American"] = pd.to_numeric(demographics_data["Native American"])
demographics_data["Asian"] = pd.to_numeric(demographics_data["Asian"])
demographics_data["Pacific Islander"] = pd.to_numeric(demographics_data["Pacific Islander"])
demographics_data["Other"] = pd.to_numeric(demographics_data["Other"])
demographics_data["Two or More"] = pd.to_numeric(demographics_data["Two or More"])
demographics_data["Hispanic"] = pd.to_numeric(demographics_data["Hispanic"])

In [16]:
demographics_data.dtypes

City                object
Total Population     int64
White                int64
African American     int64
Native American      int64
Asian                int64
Pacific Islander     int64
Other                int64
Two or More          int64
Hispanic             int64
dtype: object

Now that the columns are integers, we perform the division and replace the original columns.

In [17]:
demographics_data["White"] = demographics_data["White"] / demographics_data["Total Population"]
demographics_data["African American"] = demographics_data["African American"] / demographics_data["Total Population"]
demographics_data["Native American"] = demographics_data["Native American"] /demographics_data["Total Population"]
demographics_data["Asian"] = demographics_data["Asian"] / demographics_data["Total Population"]
demographics_data["Pacific Islander"] = demographics_data["Pacific Islander"] / demographics_data["Total Population"]
demographics_data["Other"] = demographics_data["Other"] / demographics_data["Total Population"]
demographics_data["Two or More"] = demographics_data["Two or More"]/ demographics_data["Total Population"]
demographics_data["Hispanic"] = demographics_data["Hispanic"] / demographics_data["Total Population"]

In [18]:
demographics_data[0:5]

,City,Total Population,White,African American,Native American,Asian,Pacific Islander,Other,Two or More,Hispanic
4,Agoura Hills,20330,0.843433,0.013133,0.002509,0.074816,0.001181,0.029021,0.035908,0.095229
5,Alhambra,83089,0.283082,0.015417,0.006475,0.529035,0.000975,0.130041,0.034975,0.343993
6,Arcadia,56364,0.322741,0.012082,0.003300,0.591743,0.000284,0.041729,0.028121,0.120627
7,Artesia,16522,0.390146,0.035649,0.005689,0.371081,0.002421,0.159182,0.035831,0.357705
8,Avalon,3728,0.620440,0.005365,0.005901,0.013144,0.003487,0.304989,0.046674,0.557672


Now we have the dataframe that lists each city in LA county, along with demographics information as percentages.

In [19]:
demographics_data.shape

(141, 10)

We will now join the geospatial and demographics table so that we have the cities that we have both data for.

In [20]:
cities.head()

,City,ZipCode,Latitude,Longitude
0,Manhattan Beach,90266,33.887281,-118.410607
1,Cerritos,90703,33.867227,-118.063873
2,Claremont,91711,34.095726,-117.716532
3,Burbank,91502,34.181820,-118.307890
5,Agoura Hills,91301,34.144303,-118.777612


In [21]:
result = pd.merge(cities,demographics_data,how='inner', on='City')

In [22]:
result.head()

,City,ZipCode,Latitude,Longitude,Total Population,White,African American,Native American,Asian,Pacific Islander,Other,Two or More,Hispanic
0,Manhattan Beach,90266,33.887281,-118.410607,35135,0.844912,0.008254,0.001679,0.086040,0.001395,0.011641,0.046079,0.069446
1,Cerritos,90703,33.867227,-118.063873,49041,0.231255,0.069085,0.002671,0.619135,0.002814,0.037153,0.037887,0.119961
2,Claremont,91711,34.095726,-117.716532,34926,0.706236,0.047271,0.004925,0.130676,0.001088,0.057693,0.052110,0.198105
3,Burbank,91502,34.181820,-118.307890,103340,0.727376,0.025160,0.004703,0.116189,0.000861,0.077405,0.048307,0.244920
4,Agoura Hills,91301,34.144303,-118.777612,20330,0.843433,0.013133,0.002509,0.074816,0.001181,0.029021,0.035908,0.095229


In [23]:
result.shape

(89, 13)

In [24]:
neighborhoods = result[["City","ZipCode","Latitude","Longitude"]]
neighborhoods.head()

,City,ZipCode,Latitude,Longitude
0,Manhattan Beach,90266,33.887281,-118.410607
1,Cerritos,90703,33.867227,-118.063873
2,Claremont,91711,34.095726,-117.716532
3,Burbank,91502,34.181820,-118.307890
4,Agoura Hills,91301,34.144303,-118.777612


Let's see what the geospatial data for Rosemead looks like:

In [25]:
neighborhoods.loc[neighborhoods.City=='Rosemead']

,City,ZipCode,Latitude,Longitude
59,Rosemead,91770,34.080568,-118.076757


### Foursquare Data

Now, we'll use Foursquare API to get information on restaurants in each city. We are interested in vanues in the 'food' category, and we will focus on venues that are close to the city center, i.e. the location of the city hall which we obtained earlier. Let's first obtain the dataset for Rosemead, then we will obtain the location data for remaining population after we narrow down the cities into those that are similar to Rosemead, CA in terms of demographics

Foursquare credentials are defined in hidden cell bellow.

In [1]:
# The code was removed by Watson Studio for sharing.

Let's first get the Foursquare data for the city of Rosemead. Radius is set to 1 mile from the city hall coordinates (i.e. about 1600 meters).

In [27]:
latitudes = neighborhoods.loc[59,'Latitude']
longitudes = neighborhoods.loc[59,'Longitude']

In [28]:
food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues
radius=1600
limit=200

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION, latitudes, longitudes, food_category, radius, limit)
results = requests.get(url).json()
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  if __name__ == '__main__':


In [29]:
nearby_venues[0:2]

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,...,venue.photos.count,venue.photos.groups,venue.location.neighborhood,venue.venuePage.id,venue.delivery.id,venue.delivery.url,venue.delivery.provider.name,venue.delivery.provider.icon.prefix,venue.delivery.provider.icon.sizes,venue.delivery.provider.icon.name
0,e-0-4b6b2a9bf964a52003f72be3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4b6b2a9bf964a52003f72be3,In-N-Out Burger,4242 Rosemead Blvd,at Mission Dr,34.083733,-118.073195,"[{'label': 'display', 'lat': 34.08373322493041...",...,0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,e-0-5323b498498eb004b2962377-1,0,"[{'summary': 'This spot is popular', 'type': '...",5323b498498eb004b2962377,Summer Rolls,9016 Mission,Rosemead,34.083945,-118.072719,"[{'label': 'display', 'lat': 34.08394546505231...",...,0,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
nearby_venues.shape

(100, 28)

We see that there are 100 restaurants in Rosemead, CA that are a mile from city center.

Now that we have gathered the basic data required, we are ready to perform some analysis. 